<h1 style="text-align: center;">Solving Linear Problems with <em>femtoscope<em></h1>

This notebook shows how to solve linear PDE problems with *femtoscope*.

*prerequisites* :
- FEM knowledge & knowledge on nonlinear solvers
- `mesh_generation_basics` notebook
-  `linear_problems` notebook

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

In [1]:
# Add femtoscope to the path
%reset
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Handling of nonlinear PDE problems with *femtoscope*
*femtoscope* can be used to solve nonlinear problems using Newton's method. Consider, for illustrative purposes, the following nonlinear Klein-Gordon equation
$$ \alpha \Delta u = \rho(x) - u^{-(n+1)} \quad \forall \, \mathbf{x} \in \Omega $$
Deriving the weak formulation for this PDE yields
$$ F(u, v) \coloneqq \alpha \int_{\Omega} \boldsymbol{\nabla} u \cdot \boldsymbol{\nabla} v \, \mathrm{d} \mathbf{x} - \alpha \int_{\Gamma} \boldsymbol{\nabla} u \cdot \boldsymbol{n} \, v \, \mathrm{d}\mathbf{x} + \int_{\Omega} \rho(\mathbf{x}) \, v \, \mathrm{d} \mathbf{x} - \int_{\Omega} u^{-(n+1)} v \, \mathrm{d} \mathbf{x} = 0 \ . $$

After linearizing the above weak form and applying Dirichlet boundary conditions on the boundary $\Gamma$, the weak formulation at the $k^{\mathrm{th}}$ iteration reads
$$ \alpha \int{\Omega} \boldsymbol{\nabla} u_{k+1} \cdot \boldsymbol{\nabla} v \, \mathrm{d} \mathbf{x} + (n+1) \int_{\Omega} u_{k}^{-(n+2)} u_{k+1} \, v \, \mathrm{d} \mathbf{x} - (n+2) \int_{\Omega} u_{k}^{-(n+1)} v \, \mathrm{d} \mathbf{x} + \int_{\Omega} \rho(\mathbf{x}) v \, \mathrm{d} \mathbf{x} = 0 $$

In *femtoscope*, such 'iteration-dependent'-weakforms are passed to a `NonLinearSolver` object.

#### A word about the `NonLinearSolver` class:
This class inherit from the `LinearSolver` class and overrides its `solve` method. The relaxation step can use either a constant relaxation parameter throughout all iterations, or use a more sophisticated *line-search* algorithm. The termination is managed by a separate `monitor` object of the class `NonLinearMonitor`, where stopping criteria can be defined.

## Example in the 1D case
In the spherically symmetric case, the linearized weak form reads
$$ \int_r r^2 \, u_{k+1}'(r) \, v'(r) \, \mathrm{d}r + (n+1) \int_r r^2 \, u_{k}^{-(n+2)} u_{k+1} \, v \, \mathrm{d} r - (n+2) \int_{r} r^2 \, u_{k}^{-(n+1)} v \, \mathrm{d} r + \int_{r} r^2 \, \rho(r) \, v \, \mathrm{d} r = 0 $$

We implement this weak form down below (together with the residual weak form), with
$$ \rho (r) =
\begin{cases}
    \rho_{\max} & \text{if } r \in \ [0, \, 1]\\
    \rho_{\min} & \text{if } r \in \ ]1, \, R_c]
\end{cases}
\qquad
\text{and}
\qquad
n = 2 \ .
$$

In [2]:
# Importing modules
import numpy as np
from numpy import sqrt

from femtoscope.core.pre_term import PreTerm
from femtoscope.core.weak_form import WeakForm
from femtoscope.core.solvers import NonLinearSolver
from femtoscope.core.nonlinear_monitoring import NonLinearMonitor
from femtoscope.inout.meshfactory import generate_uniform_1d_mesh

### Weak forms

In [3]:
rho_min = 1
rho_max = 1e2
alpha = 0.1
Rcut = 6.0
fem_order = 2

def create_wf_int():
    """Create the linearized weak form (instance of `WeakForm`)."""

    # Mesh creation
    pre_mesh = generate_uniform_1d_mesh(0, Rcut, 500 + 1, 'mesh_1d')

    # Terms
    def mat1(ts, coors, mode=None, **kwargs):
        if mode != 'qp': return
        val = coors.squeeze() ** 2
        return {'val': val.reshape(-1, 1, 1)}
    t1 = PreTerm('dw_laplace', mat=mat1, tag='cst', prefactor=alpha)

    def mat2(ts, coors, mode=None, vec_qp=None, **kwargs):
        if mode != 'qp': return
        val = coors.squeeze() ** 2 * vec_qp ** (-4)
        return {'val': val.reshape(-1, 1, 1)}
    t2 = PreTerm('dw_volume_dot', mat=mat2, tag='mod', prefactor=3)

    def mat3(ts, coors, mode=None, vec_qp=None, **kwargs):
        if mode != 'qp': return
        val = coors.squeeze() ** 2 * vec_qp ** (-3)
        return {'val': val.reshape(-1, 1, 1)}
    t3 = PreTerm('dw_volume_integrate', mat=mat3, tag='mod', prefactor=-4)

    def mat4(ts, coors, mode=None, **kwargs):
        if mode != 'qp': return
        r = coors.squeeze()
        rho = np.where(r <= 1.0, rho_max, rho_min)
        val = r ** 2 * rho
        return {'val': val.reshape(-1, 1, 1)}
    t4 = PreTerm('dw_volume_integrate', mat=mat4, tag='cst', prefactor=1)

    # Vertex selection
    def right_boundary(coors, domain=None):
        return np.where(coors.squeeze() == Rcut)[0]

    dim_func_entities = [(0, right_boundary, 0)]

    # WeakForm creation
    args_dict = {
        'name': 'wf_chameleon_1d',
        'dim': 1,
        'pre_mesh': pre_mesh,
        'pre_terms': [t1, t2, t3, t4],
        'dim_func_entities': dim_func_entities,
        'fem_order': fem_order,
        'pre_ebc_dict': {('vertex', 0): rho_min ** (-1 / 3)}
    }
    wf = WeakForm.from_scratch(args_dict)
    return wf

def create_wf_res():
    """Create the residual weak form (instance of `WeakForm`)."""

    # Mesh creation
    pre_mesh = generate_uniform_1d_mesh(0, Rcut, 500 + 1, 'mesh_1d')

    # Terms
    def mat1(ts, coors, mode=None, **kwargs):
        if mode != 'qp': return
        val = coors.squeeze() ** 2
        return {'val': val.reshape(-1, 1, 1)}

    t1 = PreTerm('dw_laplace', mat=mat1, tag='cst', prefactor=alpha)

    def mat2(ts, coors, mode=None, vec_qp=None, **kwargs):
        if mode != 'qp': return
        val = coors.squeeze() ** 2 * vec_qp ** (-3)
        return {'val': val.reshape(-1, 1, 1)}

    t2 = PreTerm('dw_volume_integrate', mat=mat2, tag='mod', prefactor=-1)

    def mat3(ts, coors, mode=None, **kwargs):
        if mode != 'qp': return
        r = coors.squeeze()
        rho = np.where(r <= 1, rho_max, rho_min)
        val = r ** 2 * rho
        return {'val': val.reshape(-1, 1, 1)}

    t3 = PreTerm('dw_volume_integrate', mat=mat3, tag='cst', prefactor=1)

    def mat4(ts, coors, mode=None, **kwargs):
        if mode != 'qp': return
        r = coors.squeeze()
        val = np.zeros((coors.shape[0], 1, 1))
        val[:, 0, 0] = r ** 2
        return {'val': val}

    t4 = PreTerm('dw_surface_flux', mat=mat4, tag='cst', prefactor=-alpha,
                 region_key=('vertex', 0))

    # Vertex selection
    def right_boundary(coors, domain=None):
        return np.where(coors.squeeze() == Rcut)[0]

    dim_func_entities = [(0, right_boundary, 0)]

    # WeakForm creation
    args_dict = {
        'name': 'wf_residual_1d',
        'dim': 1,
        'pre_mesh': pre_mesh,
        'pre_terms': [t1, t2, t3, t4],
        'dim_func_entities': dim_func_entities,
        'fem_order': fem_order,
    }
    wf = WeakForm.from_scratch(args_dict)
    return wf

### Solver & Monitor
The initial guess should be chosen as close to the exact solution as possible. In this example, we use our physical insights into the chameleon model and set $u_0(r) = \rho(r)^{-\frac{1}{n+1}}$. The nonlinear solver is supervised by a monitor (instance of `NonLinearMonitor`). The criteria specified down below are not active (`active=False`), but will still be computed and reported to the user (because `look=True`). The only *true* stopping criterion here is the maximum number of iterations `maximum_iter_num`.

In [4]:
def create_nonlinear_solver(wf_int, wf_res):
    wf_dict = {'wf_int': wf_int, 'wf_residual': wf_res}
    phi_min = rho_max ** (-1 / 3)
    phi_max = rho_min ** (-1 / 3)
    rr = wf_int.field.coors.squeeze()
    initial_guess = np.where(rr <= 1, phi_min, phi_max)
    initial_guess_dict = {'int': initial_guess}
    solver = NonLinearSolver(wf_dict, initial_guess_dict,
                             sol_bounds=[phi_min, phi_max])
    return solver

def create_nonlinear_monitor(nonlinear_solver):
    criteria = (
        {'name': 'RelativeDeltaSolutionNorm2', 'threshold': 1e-6, 'look': True, 'active': False},
        {'name': 'ResidualVector', 'threshold': -1, 'look': True, 'active': False},
        {'name': 'ResidualVectorNorm2', 'threshold': -1, 'look': True, 'active': False},
        {'name': 'ResidualReductionFactor', 'threshold': -1, 'look': True, 'active': False},
    )
    args_dict = {
        'minimum_iter_num': 0,
        'maximum_iter_num': 3,
        'criteria': criteria
    }
    monitor = NonLinearMonitor.from_scratch(args_dict)
    return monitor

Ultimately, the solver is linked to the monitor using the `link_monitor_to_solver` method.

In [5]:
wf_int = create_wf_int()
wf_res = create_wf_res()
solver = create_nonlinear_solver(wf_int, wf_res)
monitor = create_nonlinear_monitor(solver)
monitor.link_monitor_to_solver(solver)

# Solve Klein-Gordon equation
solver.solve(verbose=True)

_________________________ MONITORING PARAMETERS __________________________
                 criterion            look          active       threshold
         MaximumIterations            True            True               3
RelativeDeltaSolutionNorm2            True           False              -1
            ResidualVector            True           False              -1
       ResidualVectorNorm2            True           False              -1
   ResidualReductionFactor            True           False              -1
--------------------------------------------------------------------------

_____________________ ITERATION NO 1 _____________________
                 criterion           value       threshold
         MaximumIterations        1.00e+00               3
RelativeDeltaSolutionNorm2        7.49e-02              -1
       ResidualVectorNorm2        4.67e+00              -1
   ResidualReductionFactor        2.01e-01              -1
---------------------------------------------

### Re-launch iteration [feature]
Sometimes, one may want to do a few extra iterations (e.g. because convergence has not been met yet to an acceptable level). *femtoscope* makes it possible to restart iterations from where it left in only one line of code thanks to `resume` method.

In [6]:
solver.resume(force=True, new_maximum_iter_num=10)

_____________________________ ITERATION NO 3 _____________________________
                 criterion            look          active       threshold
         MaximumIterations            True            True              10
RelativeDeltaSolutionNorm2            True           False              -1
       ResidualVectorNorm2            True           False              -1
   ResidualReductionFactor            True           False              -1
--------------------------------------------------------------------------

_____________________ ITERATION NO 4 _____________________
                 criterion           value       threshold
         MaximumIterations        4.00e+00              10
RelativeDeltaSolutionNorm2        6.11e-04              -1
       ResidualVectorNorm2        1.54e-01              -1
   ResidualReductionFactor        6.65e-03              -1
----------------------------------------------------------

_____________________ ITERATION NO 5 _____________________
 